While python has built-in functions to solve ODEs, understanding how an ODE solver actually works and when it might fail is crucial to understand results obtained with these functions. In this section, we will implement standard time stepping methods, and investigate their properties.

A first important type of ODE is *Initial Value Problems* (IVP). In an IVP, we are intereted in solving equations of the form

$\frac{df}{dt}=s(f,t)$

where $f(t)$ may be a scalar or a vector, with initial condition $f(t_0)=f_0$. We note that this method is more general than it seems. If we had a higher-order ODE, e.g.

$\frac{d^2 g}{dt^2} = s(g,\frac{dg}{dt},t)$

we could define the vector $(u,v)$ with $u=g$ and $v=\frac{dg}{dt}$ and get

$\frac{du}{dt} = v$

$\frac{dv}{dt} = s(u,v,t)$

which has the same form as above, if we take $f=(u,v)$.

The simplest method to solve such a problem numerically is the *forward Euler* algorithm, where we approximate

$\frac{df}{dt} = \frac{f(t+\Delta t)-f(t)}{\Delta t}$

This allows us to write

$f(t+\Delta t) = f(t) + s(f,t) \Delta t$

Choosing a time step $\Delta t$, we can use this formula recursively to calculate $f(t_0+\Delta t)$, $f(t_0+2\Delta t)$,...

**Question 1**

Write code that uses the Forward Euler method to solve

$\frac{df}{dt} = -2 f$

with $f(0)=3.5$, and plot the result in the time interval $[0,1]$. Try using $\Delta t = 1, 0.1, 0.01, 0.001$ and compare your solution to the analytical result

$f_{\rm true} = 3.5 e^{-2t}$

In [22]:
import scipy as sp
import numpy as np
import matplotlib as plt 

def emethod(y, dt, x1, x2):
    vals = np.arange(x1,x2,dt)
    Y = np.zeros(np.size(vals) + 1)
    Y[0] = 3.5
    n = 1
    for i in vals:
        Y[n] = Y[n-1] - 2 * Y[n-1] * dt
        n = n+1
    return Y

test = emethod(3.5, .1, 0, 1)
print(test)



[3.5        2.8        2.24       1.792      1.4336     1.14688
 0.917504   0.7340032  0.58720256 0.46976205 0.37580964]


The forward Euler method is fairly inaccurate in practice: its error scales linearly with $\Delta t$. Higher-order methods are derived to have errors scaling as $(\Delta t)^n$, with $n$ the order of the method. A common example of higher order methods is the Runge-Kutta family of algorithms. A second-order Runge-Kutta (or mid-point method) follows the algorithm

$f_1 = f(t) + s(f,t) \frac{\Delta t}{2}$

$f(t+\Delta t) = f(t) + s(f_1,t) \Delta t$

to take a time step $\Delta t$. It requires two evaluations of $s$, but for sufficiently small $\Delta t$ this cost is more than offset by the higher order of convergence of the method.

**Question 2**

Repeat question 1, but with the mid-point method.

In [53]:
def mpoint(y, dt, x1, x2):
    vals = np.arange(x1,x2,dt)
    N = np.size(vals)
    f = np.zeros(N+1)
    f[0] = y
    for p in range(0,N):
        f1 = f[p] + -2*f[p]*dt*0.5
        f[p+1] = f[p] -2*f1*dt
    return f

test = mpoint(3.5, .01, 0, 1)
print(test)

[3.5        3.4307     3.36277214 3.29618925 3.2309247  3.1669524
 3.10424674 3.04278265 2.98253556 2.92348135 2.86559642 2.80885761
 2.75324223 2.69872804 2.64529322 2.59291641 2.54157667 2.49125345
 2.44192663 2.39357649 2.34618367 2.29972923 2.2541946  2.20956154
 2.16581222 2.12292914 2.08089515 2.03969342 1.99930749 1.9597212
 1.92091872 1.88288453 1.84560342 1.80906047 1.77324107 1.7381309
 1.70371591 1.66998233 1.63691668 1.60450573 1.57273652 1.54159634
 1.51107273 1.48115349 1.45182665 1.42308048 1.39490349 1.3672844
 1.34021217 1.31367597 1.28766518 1.26216941 1.23717846 1.21268232
 1.18867121 1.16513552 1.14206584 1.11945294 1.09728777 1.07556147
 1.05426535 1.0333909  1.01292976 0.99287375 0.97321485 0.9539452
 0.93505708 0.91654295 0.8983954  0.88060717 0.86317115 0.84608036
 0.82932797 0.81290728 0.79681171 0.78103484 0.76557035 0.75041206
 0.7355539  0.72098993 0.70671433 0.69272139 0.6790055  0.6655612
 0.65238308 0.6394659  0.62680447 0.61439375 0.60222875 0.59030462
 

**Question 3**

Now, let us investigate a higher order problem. Consider the equations for a ballistic trajectory with non-zero velocity,

$\frac{d^2 x}{dt^2} = - \nu \frac{dx}{dt}$

$\frac{d^2 z}{dt^2} = - \nu \frac{dz}{dt} - g$

with $\nu$ the dynamical viscosity and $g$ the gravitational acceleration. Write code allowing you to solve this problem for arbitray initial conditions and values of $(g,\nu)$. The code should terminate when the projectile hits the ground, and return the time at which this happens.

In [ ]:
def balltraj(g,v):
    

Some equations behave very poorly when using the standard time stepping methods described so far. One common issue is when the source term $s$ is "too large" (when derivatives of $s$ with respect to the evolved variables are large compared to $1/\Delta t$). We will now investigate how to deal with this issue.

**Question 4**

Try to solve the equation

$\frac{df}{dt} = 10^6 (1-f)$

for $f(0)=2.0$ with one of the methods described in questions 1-3. The correct solution is

$f_{\rm true} = (1-e^{-10^6 (t-t_0)}) + f(t_0) e^{-10^6 (t-t_0)}.$

(This type of equation is common for example in radiation transport). You will find that unless you use a very small $\Delta t$, your solution  rapidly blows up.

A potential solution is to use *implicit methods*. The general idea of an implicit method is to use $s(f,t)$ at the end of a time step when taking a time step. This of course makes our system of equations have a non-trivial dependence in $f(t+\Delta t)$. For example, the first-order method becomes

$f(t+\Delta t) - f(t) = s[f(t+\Delta t),t+\Delta t]\Delta t$

Solving this equation in general requires the use of a root-finding algorithm at each time step. For the problem of Question 4 (and any other problem linear in $f$), however, things are significantly easier. We have

$f(t+\Delta t) - f(t) = 10^6 [1-f(t+\Delta t)] \Delta t$

or

$f(t+\Delta t) = \frac{10^6 \Delta t+ f(t)}{1+10^6 \Delta t}$

**Question 5**

Use the implicit method descibed above to solve the problem from question 4, using $\Delta t = 1,0.1,0.01$. Compare your results to the true solution.

In [14]:
def solve(dt, f0, xi, xf):
    vals = np.arange(xi,xf,dt)
    N = np.size(vals)
    f = np.zeros(N+1)
    f[0] = f0
    for p in range(0,N):
        f[p+1] = ((10**6)*(dt)+f[p])/(1+(10**6)*dt)
    return f

solve(.01,2,0,1)

array([2.        , 1.00009999, 1.00000001, 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.     

Another common failure for ODE solvers, which we do not investigate here, is when $s$ goes to infinity. This may lead to the solution itself going to infinity (in which case the code should be stopped before this happens), but not necessarily: it is possible for the singularity to be integrable, i.e. for $\int dt s(t)$ to be finite. In that case, the singular point often has to be treated with a combination of analytical and numerical methods.

**Question 6**

For most practical application, it is easier to use python's built-in ODE solvers than to code your own method. To familiarize yourselves with these methods, use the odeint function from scipy to solve the problems of question 1 and 5. You may refer to the notebooks on github for example uses of odeint. Note that once you have a python setup to solve a system of differential equations using odeint, you can reuse it on any problems with simple modifications of the derivative function and initial conditions... as long as the system of equations is well-behaved!

In [30]:
def p1dfdt(f,t):
    return -2*f

def p2dfdt(f,t):
    return (10**6)*(1-f)

# sp.integrate.odeint(p1dfdt, 3.5, np.arange(0,1,0.1))
sp.integrate.odeint(p2dfdt, 2, np.arange(0,1,0.1))

array([[2.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])